identify precatalytic structures and extract a set of N frames

Structure 

1_frames 
    extract_precatalytic.ipynb
    md_run


In [1]:
# 
#   IMPORT SECTION
#
import os

from numpy import average, std
import numpy as np

#import sys
#np.set_printoptions(suppress=True, threshold=sys.maxsize)


# md analysis
from MDAnalysis import Universe
from EMDA import EMDA, __version__
from EMDA.frame_extractor import trajectory_frame_extractor
print("EMDA version is:", __version__)



# new idea
# obtain the frame index in the trajectory and select a subset of indices to perform the extraction

/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


EMDA version is: 1.0.0a5


In [2]:
# 
#   CLI ARGUMENTS
# 

# define the precatalytic definition to be used
selection = 13
dist_criterium = 5.0
dih_criterium = 60.0
lig_ID = '5-HETE' 
verbose = True
num_precat = 5

# constants
md_base_dir = os.path.abspath('./')
parent_dir = 'md_run'   
protein_ID = 'hLOX15'       

In [3]:
lig_ID_list = ['AA', '5-HpETE', '5-HETE']
if lig_ID not in lig_ID_list:
    raise ValueError(f'Introduced lig_ID {lig_ID} is not among the supported ligands: {lig_ID_list}')

# topology
topologies_1 = [i for i in os.listdir(os.path.join(md_base_dir, parent_dir)) if i.endswith('.prmtop')]
top = os.path.join(md_base_dir, parent_dir, topologies_1[0])
if len(topologies_1) > 1:
    raise ValueError('bruh. More than one topology files? Check md source directory.')
print(f'loading prmtop\t{top}')

# production trajectory
prod_dir_1 = os.path.join(md_base_dir, parent_dir, 'prod/out')                    # locate the production traj
prod_files = [int(x.strip('.nc').strip('prod_')) for x in os.listdir(prod_dir_1) if '.nc' in x]
traj_1 = [os.path.join(prod_dir_1, f'prod_{i}.nc')  for i in prod_files]      # assume constant length 250 ns, 25 
print(np.array(traj_1))

# load base MD (r=1)
emda = EMDA()
emda.load_variant(
    parameters=top,
    trajectory=traj_1,
    variant_name=protein_ID
)

u = Universe(top, traj_1)

total_time_simulated = 10 * len(traj_1)
print(f'total time simulated: {total_time_simulated} ns')

loading prmtop	/Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_extraction/md_run/h15LOX_5-HETE_343_12_mod_norelaxed_4amber_solv.prmtop
['/Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_extraction/md_run/prod/out/prod_1.nc']


/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/topology/TOPParser.py:342: UserWarning: Unknown ATOMIC_NUMBER value found for some atoms, these have been given an empty element record. If needed these can be guessed using universe.guess_TopologyAttrs(to_guess=['elements']).
  warnings.warn(msg)


hLOX15 variant has been loaded!
total time simulated: 10 ns


In [4]:
print(f'performing atom selections for {lig_ID}')

# AA
if lig_ID == lig_ID_list[0]:
    if selection == 10:
        emda.select('H10A', 'name H10A and resid 665')      # pro-S H10
        #emda.select('H10B', 'name H10B and resid 665')     # pro-R H10

        emda.select('p10_A', 'name C12 and resid 665')      # A i B formen el cis double bond
        emda.select('p10_B', 'name C11 and resid 665')
        emda.select('p10_C', 'name C10 and resid 665')      # C conté H10A i H10B
        emda.select('p10_D', 'name C9 and resid 665')       # D és el carboni més proxim al COO

    elif selection == 13:
        emda.select('H13A', 'name H13A and resid 665')      # pro-S H13
        #emda.select('H13B', 'name H13B and resid 665')     # pro-S H13

        emda.select('p13_A', 'name C11 and resid 665')      # A i B formen el cis double bond
        emda.select('p13_B', 'name C12 and resid 665')
        emda.select('p13_C', 'name C13 and resid 665')      # C conté H13A i H13B
        emda.select('p13_D', 'name C14 and resid 665')      # D és el carboni més llunyà al COO


# 5-HpETE
elif lig_ID == lig_ID_list[1]:
    if selection == 10:
        emda.select('H10A', 'name H19 and resid 665')      # pro-S H10
        #emda.select('H10B', 'name H18 and resid 665')     # pro-R H10

        emda.select('p10_A', 'name C9 and resid 665')       # A i B formen el cis double bond
        emda.select('p10_B', 'name C10 and resid 665')    
        emda.select('p10_C', 'name C11 and resid 665')      # C conté H10A i H10B
        emda.select('p10_D', 'name C12 and resid 665')      # D és el carboni més proxim al COO

    elif selection == 13:
        emda.select('H13A', 'name H15 and resid 665')      # pro-S H13
        #emda.select('H13B', 'name H14 and resid 665')     # pro-S H13

        emda.select('p13_A', 'name C10 and resid 665')      # A i B formen el cis double bond
        emda.select('p13_B', 'name C9 and resid 665')
        emda.select('p13_C', 'name C8 and resid 665')       # C conté H13A i H13B
        emda.select('p13_D', 'name C7 and resid 665')       # D és el carboni més llunyà al COO


# 5-HETE
elif lig_ID == lig_ID_list[2]:


    if selection == 10:
        emda.select('H10A', 'name H19 and resid 665')      # pro-S H10
        #emda.select('H10B', 'name H18 and resid 665')     # pro-R H10

        emda.select('p10_A', 'name C9 and resid 665')       # A i B formen el cis double bond
        emda.select('p10_B', 'name C10 and resid 665')    
        emda.select('p10_C', 'name C11 and resid 665')      # C conté H10A i H10B
        emda.select('p10_D', 'name C12 and resid 665')      # D és el carboni més proxim al COO

    elif selection == 13:
        emda.select('H13A', 'name H15 and resid 665')      # pro-S H13
        #emda.select('H13B', 'name H14 and resid 665')     # pro-S H13

        emda.select('p13_A', 'name C10 and resid 665')      # A i B formen el cis double bond
        emda.select('p13_B', 'name C9 and resid 665')
        emda.select('p13_C', 'name C8 and resid 665')       # C conté H13A i H13B
        emda.select('p13_D', 'name C7 and resid 665')       # D és el carboni més llunyà al COO

else:
    raise ValueError('Something went wrong selecting the atoms to be analyzed')

emda.select('OH1', 10552, sel_type='at_num')        # O atom from the hydroxyl cofactor
print('\n\nselected selections:\n')
print(emda.selections)


# define H10 distance and planarity
if selection == 10:

    emda.add_distance('distance H10A -- O OH1', 'OH1', 'H10A')
    #emda.add_distance('distance H10B -- O OH1', 'OH1', 'H10B')

    emda.add_dihedral('dihedral C12--C11--C10--C9 (planarity around C10)', 'p10_A', 'p10_B', 'p10_C', 'p10_D', domain=360)  

# define H13 distance and planarity
elif selection == 13:

    emda.add_distance('distance H13A -- O OH1', 'OH1', 'H13A')
    #emda.add_distance('distance H13B -- O OH1', 'OH1', 'H13B')

    emda.add_dihedral('dihedral C11--C12--C13--C14 (planarity around C13)', 'p13_A', 'p13_B', 'p13_C', 'p13_D', domain=360)

print('\n\nselected measurements:\n')
print(emda.measures)

performing atom selections for 5-HETE


selected selections:

{'H13A': 'name H15 and resid 665', 'p13_A': 'name C10 and resid 665', 'p13_B': 'name C9 and resid 665', 'p13_C': 'name C8 and resid 665', 'p13_D': 'name C7 and resid 665', 'OH1': 'bynum 10552'}


selected measurements:

{'distance H13A -- O OH1': Measure dataclass with:
	Name:   distance H13A -- O OH1
	Type:   distance
	Sel:    ['OH1', 'H13A']
	Status: 
		hLOX15, R1: Not calculated
, 'dihedral C11--C12--C13--C14 (planarity around C13)': Measure dataclass with:
	Name:   dihedral C11--C12--C13--C14 (planarity around C13)
	Type:   dihedral
	Sel:    ['p13_A', 'p13_B', 'p13_C', 'p13_D']
	Status: 
		hLOX15, R1: Not calculated
}


In [5]:
#
#   TRAJECTORY PREPROCESSING
#

precat_pkl = os.path.join(md_base_dir, f'{protein_ID}_{lig_ID}_H{selection}_precat_analysis.pkl')
try:
    # if previously computed, load the pkl file  
    print(f'loading analysis from {precat_pkl}')
    emda.load(precat_pkl)
except:
    # analyze the trajectory from scratch 
    print(f'analyzing precatalytic structures in the trajectory {parent_dir}')
    emda.run()
    emda.save(precat_pkl[:-4])    

loading analysis from /Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_extraction/hLOX15_5-HETE_H13_precat_analysis.pkl
analyzing precatalytic structures in the trajectory md_run


Measuring variant hLOX15, replica R1:   0%|          | 0/1000 [00:00<?, ? frame/s]

EMDA's measures and analyses have been saved as /Users/sergiortizropero/TFG_chem/ML-MM_setup/example/1_frames/frame_extraction/hLOX15_5-HETE_H13_precat_analysis.pkl


In [6]:
#
#   PERFORM PRECATALYTIC ANALYSIS
#
if selection == 10:
    print('analyzing C10 dist and dihedral')
    emda.analyse_value('dihedral_criterion', 'dihedral C12--C11--C10--C9 (planarity around C10)', mode='thres', val1=180-dih_criterium, val2=180+dih_criterium)
    emda.analyse_value('distance_criterion', 'distance H10A -- O OH1', mode='thres', val1=dist_criterium)
    
elif selection == 13:
    print('analyzing C13 dist and dihedral')
    emda.analyse_value('dihedral_criterion', 'dihedral C11--C12--C13--C14 (planarity around C13)', mode='thres', val1=180-dih_criterium, val2=180+dih_criterium)
    emda.analyse_value('distance_criterion', 'distance H13A -- O OH1', mode='thres', val1=dist_criterium)

else:
    raise ValueError('ep! Analysis step exploded!')


# results
criterion_1 = list(emda.analyses['dihedral_criterion'].result.values())[0]['R1']
criterion_1_idx = np.where(criterion_1)[0]
criterion_2 = list(emda.analyses['distance_criterion'].result.values())[0]['R1']
criterion_2_idx = np.where(criterion_2)[0]

# AND-type restriction
criteria = list(np.array([criterion_1, criterion_2]).min(axis=0))
criteria_idx = np.where(criteria)[0]

if verbose:

    # printing
    print(f'Dihedral criterion\n{criterion_1}\n')
    print(f'Distance criterion\n{criterion_2}\n')
    print(f'Both ({criteria.count(True)} frames)\n{criteria}\n')

analyzing C13 dist and dihedral
Dihedral criterion
[True, True, True, True, True, True, False, True, True, True, False, True, True, True, True, False, True, True, True, False, False, True, False, False, False, False, True, True, True, False, True, False, True, True, False, False, False, True, False, False, True, True, False, True, False, False, True, False, True, True, True, False, True, False, True, True, True, True, False, True, True, True, True, False, False, False, False, False, True, False, False, False, True, True, True, True, False, False, False, False, True, True, True, True, True, False, False, False, True, False, True, True, True, False, True, True, True, False, False, True, False, True, True, False, False, False, False, True, True, True, True, False, True, False, True, True, False, True, True, True, False, False, False, False, True, True, True, True, True, True, False, True, False, False, False, True, True, False, True, False, False, True, False, False, False, False, False, 

In [7]:
# final results
print(f'Frames that satisfy the dihedral criterion:\t{(len(criterion_1_idx)*100/(1000 * len(traj_1))):.2f}\t%')
print(f'Frames that satisfy the distance criterion:\t{(len(criterion_2_idx)*100/(1000 * len(traj_1))):.2f}\t%')
print(f'Frames that satisfy both criteria:\t\t{(len(criteria_idx)*100/(1000 * len(traj_1))):.2f}\t%')

Frames that satisfy the dihedral criterion:	65.40	%
Frames that satisfy the distance criterion:	3.20	%
Frames that satisfy both criteria:		3.20	%


In [8]:
# draw a random number of precatalytic structures to extract
rng = np.random.default_rng(seed=123)
random_precat_idx = criteria_idx[rng.choice(len(criteria_idx), size=num_precat, replace=False, )].tolist()
print(random_precat_idx)

[102, 542, 55, 919, 550]


In [9]:
# extract the corresponding selected frames
trajectory_frame_extractor(u, random_precat_idx, folder='frames', outformat=".pdb", solvent=True)

/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/Us